In [1]:
# Upgrade pip und installiere die benötigten Pakete
%pip install --upgrade pip
%pip install --upgrade torch transformers
import torch

#from google.colab import userdata
#userdata.get('HF_TOKEN')

# Überprüfen, ob CUDA verfügbar ist, und den Namen des CUDA-Geräts abrufen
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

import sqlite3

conn = sqlite3.connect('/content/games.db')

version = sqlite3.version
print(version)

# ...existing code...

import argparse
import torch
import re
import time
from transformers import pipeline

def create_prompt(question):  #1
    """ Generate prompt to translate question into SQL query.

    Args:
        question: question about data in natural language.

    Returns:
        prompt for question translation.
    """
    parts = []
    parts += ['Database:']
    parts += ['create table games(rank int, name text, platform text,']
    parts += ['year int, genre text, publisher text, americasales numeric,']
    parts += ['eusales numeric, japansales numeric, othersales numeric,']
    parts += ['globalsales numeric);']
    parts += ['Translate this question into SQL query:']
    parts += [question]
    return '\n'.join(parts)

def call_llm(prompt):  #2
    """ Query large language model and return answer.

    Args:
        prompt: input prompt for language model.

    Returns:
        Answer by language model.
    """
    pipe = pipeline("text-generation", model="Qwen/Qwen2-1.5B-Instruct", max_length=600, device=device, truncation=True)
    for nr_retries in range(1, 4):
        try:
            response = pipe(prompt, max_length=100)
            return response[0]['generated_text']
        except Exception as e:
            print(f"Error: {e}")
            time.sleep(nr_retries * 2)
    raise Exception('Cannot query model!')

if __name__ == '__main__':  #3
    parser = argparse.ArgumentParser()
    parser.add_argument('question', type=str, help='A question about games')
    args = parser.parse_args(['How many games are stored in total'])

    prompt = create_prompt(args.question)
    answer = call_llm(prompt) #4
    query = re.findall(r'SELECT.*?;', answer, re.DOTALL)
    if query:
        print(f'SQL: {query[0]}')
    else:
        print("No SQL query found in the response.")

cursor = conn.cursor()
cursor.execute("SELECT SUM(games.globalsales) FROM games;")  # Replace with query
results = cursor.fetchall()
print(results)

print(prompt)

print(answer)


Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   --------- ------------------------------ 2.4/10.1 MB 13.4 MB/s eta 0:00:01
   -------------------- ------------------- 5.2/10.1 MB 13.3 MB/s eta 0:00:01
   ------------------------------- -------- 7.9/10.1 MB 13.2 MB/s eta 0:00:01
   ---------------------------------------  10.0/10.1 MB 13.2 MB/s eta 0:00:01
   ---------------------------------------- 10.1/10.1 MB 11.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
Note: you may need to restart the kernel to use updated packages.


OperationalError: unable to open database file

In [ ]:

from transformers import pipeline

messages = [

            {"role": "user", "content": prompt},

           ]

pipe = pipeline("text-generation", model="Qwen/Qwen2-1.5B-Instruct", max_length=900, device='cpu')

pipe(messages)

conn = sqlite3.connect('games.db')

!pip install pathlib

import argparse
import torch
import re
import time
from transformers import pipeline
import sqlite3
import sys
from pathlib import Path

# Connect to the database
#conn = sqlite3.connect('games.db')
#cursor = conn.cursor()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_structure(data_path): #1 Extracts the database structure
    """ Extract structure from SQLite database.

    Args:
        data_path: path to SQLite data file.

    Returns:
        text description of database structure.
    """
    with sqlite3.connect(data_path) as connection:
        cursor = connection.cursor()
        cursor.execute("select sql from sqlite_master where type = 'table';")
        table_rows = cursor.fetchall()
        table_ddls = [r[0] for r in table_rows]
    return '\n'.join(table_ddls)

def create_prompt(description, question): #2 Creates a prompt for translation
    """ Generate prompt to translate a question into an SQL query.

    Args:
        description: text description of database structure.
        question: question about data in natural language.

    Returns:
        prompt for question translation.
    """
    parts = []
    parts += ['Database:']
    parts += [description]
    parts += ['Translate this question into SQL query:']
    parts += [question]
    return '\n'.join(parts)

def call_llm(prompt): #3 Invokes the language model
    """ Query large language model and return answer.

    Args:
        prompt: input prompt for language model.

    Returns:
        Answer by language model.
    """
    pipe = pipeline("text-generation", model="Qwen/Qwen2-1.5B-Instruct", max_length=800, device=device, truncation=True, temperature=0.7)

    for nr_retries in range(1, 4):
        try:
            response = pipe(prompt, max_length=800)
            return response[0]['generated_text']
        except:
            time.sleep(nr_retries * 2)
    raise Exception('Cannot query model!')

def process_query(data_path, query): #4 Processes a query on a database
    """ Processes SQL query and returns result.

    Args:
        data_path: path to SQLite data file.
        query: process this query on database.

    Returns:
        query result.
    """
    with sqlite3.connect(data_path) as connection:
        cursor = connection.cursor()
        cursor.execute(query)
        table_rows = cursor.fetchall()
        table_strings = [str(r) for r in table_rows]
        return '\n'.join(table_strings)

if __name__ == '__main__':
    ### Simulate command-line arguments
    ###sys.argv = ['ipykernel_launcher.py', r'C:\\Users\\kappv\\Desktop\\Data-Analysis-with-LLM-3\\games.db']
    parser = argparse.ArgumentParser()
    parser.add_argument('dbpath', type=str, help='Path to SQLite data')
    # Instead of directly calling parse_args(), provide the path to the database
    #args = parser.parse_args() # original
    args = parser.parse_args(['games.db']) # updated call using database path

    data_structure = get_structure(args.dbpath) #5 Reads data structure
    while True: #6  Answers questions until the user quits
        user_input = input('Enter question:')
        if user_input == 'quit':
            break
        prompt = create_prompt(data_structure, user_input)
        answer = call_llm(prompt)
        query = re.findall('SELECT.*?;', answer, re.DOTALL)[0]
        print(f'SQL: {query}')

        try: #7  Processes query on the database
            result = process_query(args.dbpath, query)
            print(f'Result: {result}')
        except:
            print('Error processing query! Try to reformulate.')

print("LLM Response:", answer)

query = re.findall(r'SELECT.*?;', answer, re.DOTALL)
if query:
    query = query[0]
else:
    raise ValueError("No SQL query found in the response.")

print(query)